In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import numpy as np
from sklearn import svm

D:\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#global variabls

directory_path = "D:/MLOntology/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"

In [3]:
conceptLabelDict={}
errors=[]

def read_label(fname):
    conceptLabelDict={}
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptLabelDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)
    return conceptLabelDict

label_file_2017 = data_path + "ontClassLabels_july2017.txt"
conceptLabelDict_2017=read_label(label_file_2017)
label_file_2018= data_path + "ontClassLabels_jan2018.txt"
conceptLabelDict_2018=read_label(label_file_2018)

In [8]:
import json
from pprint import pprint

jsonFile = data_path + "2018newconcepts.json"

data = json.load(open(jsonFile))

# pprint(data)

# pprint(data['100841000119109']['Ancestors'][1])

In [9]:
def readFromJsonData(data):
    result_paired = []
    result_not_paired= []
    for key, value in data.items():
        if value['Parents']:
            for x in range(len(value['Parents'])):
                result_paired.append([key, value['Parents'][x], 1])
        if value['Siblings']:
            for x in range(len(value['Siblings'])):
                result_not_paired.append([key, value['Siblings'][x], 0])
        if value['Children']:
            for x in range(len(value['Children'])):
                result_not_paired.append([key, value['Children'][x], 0])
    return result_paired, result_not_paired
    
    

In [10]:
paired_list, not_paired_list = readFromJsonData(data)
pprint(paired_list[:20])
pprint(not_paired_list[:20])

[['1191000087104', '449922004', 1],
 ['736477006', '362981000', 1],
 ['735266006', '115070006', 1],
 ['736236009', '42993004', 1],
 ['737447009', '23468006', 1],
 ['737447009', '726439007', 1],
 ['737562008', '369071000119105', 1],
 ['737562008', '253883006', 1],
 ['736110006', '708115005', 1],
 ['735942009', '264274002', 1],
 ['735942009', '744854002', 1],
 ['737206009', '301811001', 1],
 ['737206009', '7066000', 1],
 ['15935061000119100', '240254005', 1],
 ['15935061000119100', '16010991000119100', 1],
 ['15935061000119100', '203541003', 1],
 ['735025000', '36946006', 1],
 ['735381003', '38850007', 1],
 ['735381003', '41888000', 1],
 ['734972007', '386952008', 1]]
[['1191000087104', '425848005', 0],
 ['1191000087104', '241511008', 0],
 ['1191000087104', '1181000087101', 0],
 ['1191000087104', '303926006', 0],
 ['1191000087104', '1041000087105', 0],
 ['1191000087104', '1171000087103', 0],
 ['1191000087104', '14301000087106', 0],
 ['1191000087104', '1081000087100', 0],
 ['1191000087104

In [11]:
vector_model_file = vector_model_path + "model0"

vector_model = gensim.models.Doc2Vec.load(vector_model_file)

inferred_vector = vector_model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(vector_model.docvecs.most_similar([inferred_vector], topn=10))

[('237267007', 0.585956871509552),
 ('722912007', 0.5496871471405029),
 ('722913002', 0.5403156280517578),
 ('446466006', 0.5247969627380371),
 ('10759711000119103', 0.515601634979248),
 ('267262008', 0.5108802318572998),
 ('10759661000119108', 0.5068628191947937),
 ('177130000', 0.5061954855918884),
 ('12729009', 0.5008457899093628),
 ('10759611000119105', 0.5008142590522766)]


In [20]:
import tensorflow as tf
 
sess=tf.Session()    
#First let's load meta graph and restore weights
cnn_model_file = cnn_model_path + 'model-noleaky.ckpt.meta'
saver = tf.train.import_meta_graph(cnn_model_file)
saver.restore(sess,tf.train.latest_checkpoint(cnn_model_path))

INFO:tensorflow:Restoring parameters from D:/MLOntology/cnnModel/model-noleaky.ckpt


In [21]:
graph = tf.get_default_graph()
x = graph.get_tensor_by_name("input_vector:0")
y = graph.get_tensor_by_name("class_label:0")
keep_prob = tf.placeholder(tf.float32)
predict= graph.get_tensor_by_name("predict:0")
correct_prediction= graph.get_tensor_by_name("correct_prediction:0")
accuracy= graph.get_tensor_by_name("accuracy:0")

In [ ]:
# correct_prediction = tf.equal(predict, tf.argmax(y,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"), name="accuracy")

In [22]:
feature_number = 1024

def processConceptParents(json_data):
    parents_dict = {}
    for key, value in json_data.items():
        parents_list = []
        if value['Parents']:
            for x in range(len(value['Parents'])):
                parents_list.append(value['Parents'][x])
        parents_dict[key]= parents_list   
    return parents_dict




def getInferredVector(concept_id, conceptLabelDict, model, op_str = ''):
    if concept_id in model.docvecs:
        concept_vector= model.docvecs[concept_id]
    else:
        concept_label = conceptLabelDict[concept_id]
        concept_vector= model.infer_vector(concept_label.split() + op_str.split())
    return concept_vector


def getVectorsFromModel(pair_list, conceptLabelDict, model, opt_str=""):
    vector_list =[]
    label_list =[]
    for i, line in enumerate(pair_list): 
        a= getInferredVector(line[0], conceptLabelDict, model, opt_str)
        b= getInferredVector(line[1], conceptLabelDict, model)
        c = np.array((a, b))
#         test_list.append(np.reshape(c, feature_number))
        vector_list.append(np.reshape(c, feature_number, order='F'))
        label_list.append(line[2])
    return vector_list, label_list

         

def readOutPairs(key, parent, paired_list, not_paired_list):
    pair_list=[]
    for i in range(len(paired_list)):
        pair=paired_list[i]
        if pair[0] == key and pair[1]!= parent:
            pair_list.append(pair)
    for i in range(len(not_paired_list)):
        pair=not_paired_list[i]
        if pair[0] == key:
            pair_list.append(pair)
    return pair_list

In [28]:
parents_dict = processConceptParents(data)

total_predictions_count = 0
correct_predictions_count = 0

for key, parents in parents_dict.items():
    print("Processing key ", key)
    if len(parents) > 1:
        for i in range(len(parents)):            
            parent = parents[i]
            print("\tProcessing its %d parent %s "%(i, parent))
            if parent in conceptLabelDict_2017:
                parent_str = conceptLabelDict_2017[parent]
            else:
                parent_str = conceptLabelDict_2018[parent]
            pair_list = readOutPairs(key, parent, paired_list, not_paired_list)
            vector_list, label_list = getVectorsFromModel(pair_list, conceptLabelDict_2018, vector_model, parent_str)
            y_pred = sess.run(predict, feed_dict={x:vector_list, keep_prob:1})
            err_ids = np.flatnonzero(y_pred != label_list)
            print("\t\t %d predictions Correct: %s" % (len(y_pred),len(err_ids)))
            total_predictions_count += len(y_pred)
            correct_predictions_count += len(err_ids)
#             acc = sess.run(accuracy, feed_dict={x:vector_list, y: np.eye(2)[label_list], keep_prob:1})
#             print("\t\t %d predictions Acc: %s" % (len(pair_list),acc))
                              
            

Processing key  1191000087104
Processing key  736477006
Processing key  735266006
Processing key  736236009
Processing key  737447009
	Processing its 0 parent 23468006 
		 1 predictions Correct: 0
	Processing its 1 parent 726439007 
		 1 predictions Correct: 1
Processing key  737562008
	Processing its 0 parent 369071000119105 
		 2 predictions Correct: 2
	Processing its 1 parent 253883006 
		 2 predictions Correct: 2
Processing key  736110006
Processing key  735942009
	Processing its 0 parent 264274002 
		 1 predictions Correct: 0
	Processing its 1 parent 744854002 
		 1 predictions Correct: 1
Processing key  737206009
	Processing its 0 parent 301811001 
		 4 predictions Correct: 3
	Processing its 1 parent 7066000 
		 4 predictions Correct: 3
Processing key  15935061000119100
	Processing its 0 parent 240254005 
		 3 predictions Correct: 1
	Processing its 1 parent 16010991000119100 
		 3 predictions Correct: 2
	Processing its 2 parent 203541003 
		 3 predictions Correct: 2
Processing ke

		 2 predictions Correct: 2
Processing key  72271000119100
Processing key  736490004
	Processing its 0 parent 27879009 
		 2 predictions Correct: 2
	Processing its 1 parent 119590004 
		 2 predictions Correct: 2
	Processing its 2 parent 389059002 
		 2 predictions Correct: 2
Processing key  735140000
Processing key  734514009
Processing key  6421000087103
Processing key  738532000
Processing key  762415003
	Processing its 0 parent 301934009 
		 12 predictions Correct: 11
	Processing its 1 parent 302217008 
		 12 predictions Correct: 10
Processing key  761904000
Processing key  294031000119103
Processing key  735050002
Processing key  15978631000119109
	Processing its 0 parent 292631000119102 
		 1 predictions Correct: 0
	Processing its 1 parent 292621000119100 
		 1 predictions Correct: 0
Processing key  734821002
Processing key  660441000119109
Processing key  737357006
	Processing its 0 parent 119297000 
		 2 predictions Correct: 1
	Processing its 1 parent 309502007 
		 2 predictions

		 4 predictions Correct: 4
	Processing its 3 parent 426730003 
		 4 predictions Correct: 1
	Processing its 4 parent 709767008 
		 4 predictions Correct: 3
Processing key  736936003
Processing key  737231005
	Processing its 0 parent 230711001 
		 3 predictions Correct: 3
	Processing its 1 parent 127294003 
		 3 predictions Correct: 3
	Processing its 2 parent 127296001 
		 3 predictions Correct: 3
	Processing its 3 parent 128126004 
		 3 predictions Correct: 3
Processing key  762275008
Processing key  737141002
Processing key  427561000119104
	Processing its 0 parent 432054008 
		 3 predictions Correct: 3
	Processing its 1 parent 416608005 
		 3 predictions Correct: 3
	Processing its 2 parent 427541000119103 
		 3 predictions Correct: 3
Processing key  737078002
Processing key  762836006
	Processing its 0 parent 758646000 
		 2 predictions Correct: 0
	Processing its 1 parent 71065005 
		 2 predictions Correct: 0
	Processing its 2 parent 758682005 
		 2 predictions Correct: 0
Processing 

		 5 predictions Correct: 3
Processing key  737280000
	Processing its 0 parent 118236001 
		 1 predictions Correct: 1
	Processing its 1 parent 397578001 
		 1 predictions Correct: 1
Processing key  3111000087106
	Processing its 0 parent 2611000087108 
		 3 predictions Correct: 0
	Processing its 1 parent 3131000087101 
		 3 predictions Correct: 0
	Processing its 2 parent 3101000087109 
		 3 predictions Correct: 0
	Processing its 3 parent 2591000087103 
		 3 predictions Correct: 0
Processing key  762681007
	Processing its 0 parent 473040003 
		 2 predictions Correct: 2
	Processing its 1 parent 362977000 
		 2 predictions Correct: 2
Processing key  734488002
Processing key  735737009
	Processing its 0 parent 206331005 
		 1 predictions Correct: 1
	Processing its 1 parent 88594005 
		 1 predictions Correct: 1
Processing key  737943000
Processing key  15978711000119106
Processing key  16302031000119101
	Processing its 0 parent 128135006 
		 6 predictions Correct: 4
	Processing its 1 parent 

		 5 predictions Correct: 5
	Processing its 1 parent 429554009 
		 5 predictions Correct: 5
	Processing its 2 parent 298149009 
		 5 predictions Correct: 4
	Processing its 3 parent 35524003 
		 5 predictions Correct: 4
Processing key  16260551000119106
	Processing its 0 parent 20301004 
		 4 predictions Correct: 4
	Processing its 1 parent 1131000119105 
		 4 predictions Correct: 4
Processing key  736900001
	Processing its 0 parent 16349000 
		 2 predictions Correct: 1
	Processing its 1 parent 466151005 
		 2 predictions Correct: 2
Processing key  735328006
Processing key  734665008
Processing key  740515000
	Processing its 0 parent 116140006 
		 2 predictions Correct: 2
	Processing its 1 parent 176917009 
		 2 predictions Correct: 2
Processing key  735687006
Processing key  734235005
Processing key  736081002
Processing key  762566005
Processing key  734923000
Processing key  735214002
Processing key  413431000119108
Processing key  144511000119107
Processing key  737017003
	Processing

...

		 3 predictions Correct: 1
Processing key  738513006
Processing key  2991000087109
	Processing its 0 parent 2471000087103 
		 3 predictions Correct: 1
	Processing its 1 parent 3021000087102 
		 3 predictions Correct: 1
	Processing its 2 parent 2491000087104 
		 3 predictions Correct: 1
Processing key  736863001
Processing key  286571000119101
Processing key  735488008
	Processing its 0 parent 198130006 
		 7 predictions Correct: 7
	Processing its 1 parent 116223007 
		 7 predictions Correct: 7
	Processing its 2 parent 609620004 
		 7 predictions Correct: 6
	Processing its 3 parent 128070006 
		 7 predictions Correct: 7
	Processing its 4 parent 363165003 
		 7 predictions Correct: 7
	Processing its 5 parent 363296001 
		 7 predictions Correct: 6
Processing key  329491000119109
	Processing its 0 parent 118934005 
		 12 predictions Correct: 11
	Processing its 1 parent 2929001 
		 12 predictions Correct: 11
	Processing its 2 parent 250845006 
		 12 predictions Correct: 11
	Processing its 

		 4 predictions Correct: 4
	Processing its 1 parent 22811006 
		 4 predictions Correct: 4
	Processing its 2 parent 85995004 
		 4 predictions Correct: 4
	Processing its 3 parent 363235000 
		 4 predictions Correct: 4
	Processing its 4 parent 240096000 
		 4 predictions Correct: 4
Processing key  735249009
	Processing its 0 parent 406465008 
		 6 predictions Correct: 1
	Processing its 1 parent 227416001 
		 6 predictions Correct: 2
Processing key  734680002
Processing key  736655000
Processing key  412361000119103
	Processing its 0 parent 303786002 
		 2 predictions Correct: 2
	Processing its 1 parent 69064006 
		 2 predictions Correct: 2
	Processing its 2 parent 168821007 
		 2 predictions Correct: 2
Processing key  737120008
Processing key  742277008
	Processing its 0 parent 741676008 
		 5 predictions Correct: 4
	Processing its 1 parent 741677004 
		 5 predictions Correct: 3
Processing key  16459631000119104
	Processing its 0 parent 151000087108 
		 3 predictions Correct: 1
	Process

	Processing its 2 parent 299128000 
		 3 predictions Correct: 3
	Processing its 3 parent 429340002 
		 3 predictions Correct: 3
Processing key  734254005
Processing key  736083004
Processing key  15985831000119107
	Processing its 0 parent 118933004 
		 2 predictions Correct: 2
	Processing its 1 parent 445006008 
		 2 predictions Correct: 2
Processing key  17291000087101
	Processing its 0 parent 151000087108 
		 3 predictions Correct: 2
	Processing its 1 parent 4711000087109 
		 3 predictions Correct: 3
	Processing its 2 parent 419870000 
		 3 predictions Correct: 1
Processing key  1077731000119109
Processing key  734667000
Processing key  737237009
	Processing its 0 parent 262702004 
		 1 predictions Correct: 1
	Processing its 1 parent 721428008 
		 1 predictions Correct: 1
Processing key  735591005
	Processing its 0 parent 46876003 
		 3 predictions Correct: 3
	Processing its 1 parent 85189001 
		 3 predictions Correct: 3
	Processing its 2 parent 127321000 
		 3 predictions Correct: 3

		 1 predictions Correct: 1
	Processing its 1 parent 123250005 
		 1 predictions Correct: 1
Processing key  1078041000119104
Processing key  762372009
Processing key  740602002
	Processing its 0 parent 2033006 
		 1 predictions Correct: 0
	Processing its 1 parent 245499003 
		 1 predictions Correct: 1
Processing key  734878005
Processing key  737846009
	Processing its 0 parent 409057001 
		 36 predictions Correct: 2
	Processing its 1 parent 408766003 
		 36 predictions Correct: 2
Processing key  745273003
Processing key  762319005
	Processing its 0 parent 62305002 
		 2 predictions Correct: 2
	Processing its 1 parent 280032002 
		 2 predictions Correct: 2
Processing key  739075009
	Processing its 0 parent 739074008 
		 1 predictions Correct: 0
	Processing its 1 parent 739066007 
		 1 predictions Correct: 0
Processing key  762664003
	Processing its 0 parent 403219000 
		 1 predictions Correct: 1
	Processing its 1 parent 238513004 
		 1 predictions Correct: 1
Processing key  735858008
	P

		 1 predictions Correct: 1
	Processing its 1 parent 274172008 
		 1 predictions Correct: 1
Processing key  738984000
Processing key  736891009
	Processing its 0 parent 397968008 
		 1 predictions Correct: 1
	Processing its 1 parent 708115005 
		 1 predictions Correct: 1
Processing key  292501000119108
Processing key  736031006
Processing key  328271000119104
Processing key  737448004
	Processing its 0 parent 23468006 
		 1 predictions Correct: 0
	Processing its 1 parent 726438004 
		 1 predictions Correct: 1
Processing key  736785003
Processing key  736229008
Processing key  734615004
Processing key  735379000
	Processing its 0 parent 265780004 
		 1 predictions Correct: 0
	Processing its 1 parent 735084003 
		 1 predictions Correct: 0
Processing key  736972005
	Processing its 0 parent 232721002 
		 1 predictions Correct: 1
	Processing its 1 parent 175021005 
		 1 predictions Correct: 1
Processing key  16554511000119103
Processing key  736112003
Processing key  738404007
Processing ke

In [39]:
print("There are %d predictions \t correct predictions: %d" % (total_predictions_count,correct_predictions_count)) 
print("Total accuracy is %f" % (correct_predictions_count/total_predictions_count))

There are 21945 predictions 	 correct predictions: 14917
Total accuracy is 0.679745


In [ ]:
y_pred = sess.run(predict, feed_dict={x:test_feature_batch, keep_prob:1, is_training:False})
err_ids=np.flatnonzero(np.eye(2)[y_pred] != test_y_m_batch)
for err_id in err_ids:
    print("index %d predicted label %s, but true label is %s" % (err_id, y_pred[err_id], test_y_m_batch[err_id]))
    idpair = ids_test[err_id] 
    concept1 = conceptLabelDict[idpair[0]]
    concept2 = conceptLabelDict[idpair[1]]
    print("%s Concept Pairs: (%s --- %s)" % (idpair, concept1, concept2 ))